In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
from sklearn import svm
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import os
import numpy as np

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
dir_img = "E:\GOS\data\CatDog"
dir_train = os.path.join(dir_img, "training_set\\training_set")
dir_test = os.path.join(dir_img, "test_set\\test_set")

In [29]:
dir_train

'E:\\GOS\\data\\CatDog\\training_set\\training_set'

In [4]:
resnet = models.resnet50(pretrained=True)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Loại bỏ lớp fully connected cuối cùng
modules = list(resnet.children())[:-1]  # Bỏ layer fc
resnet = torch.nn.Sequential(*modules)

In [8]:
Transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
resnet.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [10]:
# Trích xuất đặc trưng từ hình ảnh
def Feature_Extraction(path):
  features=[]
  labels=[]
  for dir_name in os.listdir(path):
      label_name =  dir_name
      dir_class = os.path.join(path,label_name)
      for file_name in os.listdir(dir_class):
        path_img = os.path.join(dir_class,file_name)
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
        img = Image.open(path_img)
        img = Transforms(img)
        batch = torch.unsqueeze(img, 0)
        with torch.no_grad():
          fc7_features = resnet(batch)
          features.append(fc7_features.squeeze())
        if (label_name == 'cats'):
          labels.append(0)
        elif (label_name == 'dogs'):
          labels.append(1)
  return features, labels

In [32]:
feature_train, label_train = Feature_Extraction(dir_train)
# feature_val, label_val = Feature_Extraction(dir_validate)
feature_test, label_test = Feature_Extraction(dir_test)

In [33]:
import os
print("Tồn tại thư mục?", os.path.exists(dir_train))

Tồn tại thư mục? True


In [34]:
feature_train = np.array(feature_train)
label_train = np.array(label_train)
feature_test = np.array(feature_test)
label_test = np.array(label_test)

In [35]:
feature_train

array([[0.10589694, 0.02697366, 0.00606085, ..., 0.17974333, 0.1681424 ,
        0.06847233],
       [0.908417  , 0.16832389, 0.07923365, ..., 0.5355617 , 0.3064112 ,
        0.2037155 ],
       [0.42056665, 0.4217158 , 0.25521916, ..., 0.43453598, 0.3126373 ,
        0.597157  ],
       ...,
       [1.3053623 , 0.21633804, 0.457754  , ..., 0.16366665, 0.81825644,
        0.07632446],
       [0.277001  , 0.41414008, 0.40006852, ..., 0.6872395 , 0.5001063 ,
        0.16033906],
       [0.12874596, 0.5156523 , 0.39731103, ..., 0.22563598, 0.32591075,
        0.17528611]], dtype=float32)

In [36]:
array_train = feature_train.tolist()

In [37]:
array_test = feature_test.tolist()

In [57]:
torch.save(resnet, 'resnet50_feature_extractor.pth')

In [38]:
svm_model = svm.SVC(kernel='linear', C=1.0)

# Huấn luyện mô hình SVM trên tập dữ liệu huấn luyện và nhãn tương ứng
svm_model.fit(array_train, label_train)  # train_features là đặc trưng, train_labels là nhãn của tập huấn luyện

# Dự đoán trên tập dữ liệu kiểm tra
predictions = svm_model.predict(array_test)

In [55]:
import joblib

joblib.dump(svm_model, "model.pkl")

['model.pkl']

In [40]:
from sklearn.metrics import classification_report
target_names = ['cats', 'dogs']
print(classification_report(label_test,predictions, target_names=target_names))

              precision    recall  f1-score   support

        cats       0.98      0.99      0.99      1011
        dogs       0.99      0.98      0.99      1012

    accuracy                           0.99      2023
   macro avg       0.99      0.99      0.99      2023
weighted avg       0.99      0.99      0.99      2023



In [ ]:
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt

# cm = confusion_matrix(label_test,predictions)
# plt.imshow(cm, interpolation='nearest', cmap=plt.get_cmap('Blues'))
# plt.title('Confusion Matrix')
# plt.colorbar()

# for i in range(len(cm)):
#     for j in range(len(cm[i])):
#         plt.text(j, i, str(cm[i][j]), ha='center', va='center', color='black', fontsize=14)

# tick_marks = np.arange(len(cm))
# plt.xticks(tick_marks, ['Class 0', 'Class 1'])
# plt.yticks(tick_marks, ['Class 0', 'Class 1'])
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()

In [42]:
def Feature_Extraction_img(img_path):
    features=[]
    file_name = os.path.basename(img_path)
    if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')): return
    img = Image.open(img_path)
    img = Transforms(img)
    batch = torch.unsqueeze(img, 0)
    with torch.no_grad():
      fc7_features = resnet(batch)
      features.append(fc7_features.squeeze())
    return features

In [52]:
img_path = "E:/GOS/data/CatDog/test_set/test_set/cats/cat.4652.jpg"
img_test = Feature_Extraction_img(img_path)

In [56]:
img_test

[tensor([0.2506, 0.1544, 0.1386,  ..., 0.6124, 0.2649, 0.0116])]

In [53]:
predictions = svm_model.predict(img_test)

In [54]:
if predictions[0] == 1:
  print("dog")
else: print("cat")

cat
